In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


In [ ]:
import cv2
import os

def test_video_processing(video_path):
    print(f"Attempting to open video: {video_path}")
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Failed to open video: {video_path}")
        return
    else:
        print(f"Video opened successfully: {video_path}")

    ret, frame = cap.read()
    if not ret:
        print("Failed to read the first frame.")
        return
    else:
        print("First frame read successfully.")

    # Test saving the first frame
    test_output_path = 'test_frame.jpg'
    if cv2.imwrite(test_output_path, frame):
        print(f"Frame saved to {test_output_path}")
    else:
        print("Failed to save frame.")

    cap.release()

if __name__ == "__main__":
    # Update this path to a specific video file to test
    test_video_path = '/content/drive/MyDrive/Raw_Processing/Vid/Copy of 3_l90.MOV'
    if os.path.exists(test_video_path):
        test_video_processing(test_video_path)
    else:
        print(f"Video file does not exist: {test_video_path}")


Attempting to open video: /content/drive/MyDrive/Raw_Processing/Vid/Copy of 3_l90.MOV
Video opened successfully: /content/drive/MyDrive/Raw_Processing/Vid/Copy of 3_l90.MOV
First frame read successfully.
Frame saved to test_frame.jpg


In [ ]:
import os

def delete_video_files(directory, extensions):
    # Walk through all directories and files in the specified directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file extension is in the list of video extensions and does not contain "00" or "l45"
            if file.endswith(tuple(extensions)) and "00" not in file and "l45" not in file:
                # Construct the full file path
                file_path = os.path.join(root, file)
                # Delete the file
                os.remove(file_path)
                print(f"Deleted: {file_path}")

# Specify the directory to search in
input_directory = '/content/drive/MyDrive/data/videos'
# Specify the video file extensions to delete
video_extensions = ['MP4', 'avi', 'MOV', 'mkv']

# Call the function
delete_video_files(input_directory, video_extensions)


Deleted: /content/drive/MyDrive/dataset5/videos/6/6_l90.MOV
Deleted: /content/drive/MyDrive/dataset5/videos/6/6_r90.MOV
Deleted: /content/drive/MyDrive/dataset5/videos/6/6_r45.MOV
Deleted: /content/drive/MyDrive/dataset5/videos/3/3_l90.MOV
Deleted: /content/drive/MyDrive/dataset5/videos/3/3_r90.MOV
Deleted: /content/drive/MyDrive/dataset5/videos/3/3_r45.MOV
Deleted: /content/drive/MyDrive/dataset5/videos/26/26_l90.MOV
Deleted: /content/drive/MyDrive/dataset5/videos/26/26_r90.MOV
Deleted: /content/drive/MyDrive/dataset5/videos/26/26_r45.MOV
Deleted: /content/drive/MyDrive/dataset5/videos/9/9_l90.MOV
Deleted: /content/drive/MyDrive/dataset5/videos/9/9_r45.MOV
Deleted: /content/drive/MyDrive/dataset5/videos/9/9_r90.MOV
Deleted: /content/drive/MyDrive/dataset5/videos/4/4_r90.MOV
Deleted: /content/drive/MyDrive/dataset5/videos/4/4_r45.MOV
Deleted: /content/drive/MyDrive/dataset5/videos/4/4_l90.MOV
Deleted: /content/drive/MyDrive/dataset5/videos/27/27_l90.MOV
Deleted: /content/drive/MyDrive/

In [ ]:
import cv2
import os

Dataset_path = '/content/drive/MyDrive/dataset5/videos'
for folder in os.listdir(Dataset_path):
    folder_path = os.path.join(Dataset_path, folder)
    if os.path.isdir(folder_path):  # Check if it is a directory
        for file_path in os.listdir(folder_path):
            # Define full input video path
            input_video_path = os.path.join(folder_path, file_path)

            # Get the last folder name
            last_folder_name = os.path.basename(folder_path)
            # Create a subfolder based on the last folder name
            output_directory = os.path.join("/content/drive/MyDrive/dataset5/frames", last_folder_name)
            os.makedirs(output_directory, exist_ok=True)

            # Open the video file
            cap = cv2.VideoCapture(input_video_path)
            if not cap.isOpened():
                print(f"Failed to open video {input_video_path}")
                continue

            # Get the total number of frames in the video
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            fps = cap.get(cv2.CAP_PROP_FPS)

            # Determine fps based on video title
            target_fps = 60 if 'l45' in file_path else 24
            frame_interval = int(round(fps / target_fps))

            # Create a directory for this video
            video_name = os.path.splitext(file_path)[0]  # Extract video name without extension
            video_output_dir = os.path.join(output_directory, video_name)
            os.makedirs(video_output_dir, exist_ok=True)

            frame_count = 0
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                # Skip frames
                if frame_count % frame_interval != 0:
                    frame_count += 1
                    continue

                # Convert the BGR image to RGB and preprocess
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # Save the preprocessed frame as an image
                output_path = os.path.join(video_output_dir, f'frame_{frame_count // frame_interval}.jpg')
                cv2.imwrite(output_path, frame_rgb)

                frame_count += 1

            # Release resources
            cap.release()
            print(f"All frames from video '{video_name}' processed and saved successfully.")
    else:
        print(f"Skipping file (not a directory): {folder_path}")


All frames from video '6_00' processed and saved successfully.
All frames from video '6_l45' processed and saved successfully.
All frames from video '3_l45' processed and saved successfully.
All frames from video '3_00' processed and saved successfully.
All frames from video '26_l45' processed and saved successfully.
All frames from video '26_00' processed and saved successfully.
All frames from video '9_00' processed and saved successfully.
All frames from video '9_l45' processed and saved successfully.
All frames from video '4_00' processed and saved successfully.
All frames from video '4_l45' processed and saved successfully.
All frames from video '27_l45' processed and saved successfully.
All frames from video '27_00' processed and saved successfully.
All frames from video '25_l45' processed and saved successfully.
All frames from video '25_00' processed and saved successfully.
All frames from video '5_00' processed and saved successfully.
All frames from video '5_l45' processed an

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import os
import matplotlib.pyplot as plt

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def calculate_angle(p1, p2, p3):
    v1 = np.array(p1) - np.array(p2)
    v2 = np.array(p3) - np.array(p2)
    v1_normalized = v1 / np.linalg.norm(v1)
    v2_normalized = v2 / np.linalg.norm(v2)
    angle = np.arccos(np.clip(np.dot(v1_normalized, v2_normalized), -1.0, 1.0))
    return np.degrees(angle)

def calculate_distance(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

def get_landmarks(landmarks, frame_shape):
    left_heel = (int(landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x * frame_shape[1]),
                 int(landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y * frame_shape[0]))
    left_toe = (int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x * frame_shape[1]),
                int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y * frame_shape[0]))
    right_heel = (int(landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x * frame_shape[1]),
                  int(landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y * frame_shape[0]))
    right_toe = (int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x * frame_shape[1]),
                 int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y * frame_shape[0]))
    return left_heel, left_toe, right_heel, right_toe

def draw_connections(image, positions):
    pose_connections = [
        (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER),
        (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW),
        (mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
        (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW),
        (mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST),
        (mp_pose.PoseLandmark.LEFT_ANKLE, mp_pose.PoseLandmark.LEFT_HEEL),
        (mp_pose.PoseLandmark.LEFT_HEEL, mp_pose.PoseLandmark.LEFT_FOOT_INDEX),
        (mp_pose.PoseLandmark.RIGHT_ANKLE, mp_pose.PoseLandmark.RIGHT_HEEL),
        (mp_pose.PoseLandmark.RIGHT_HEEL, mp_pose.PoseLandmark.RIGHT_FOOT_INDEX),
    ]
    for connection in pose_connections:
        start_point = positions[connection[0]]
        end_point = positions[connection[1]]
        cv2.line(image, start_point, end_point, (255, 0, 0), 3)

def get_joint_positions(landmarks, frame_shape):
    positions = {}
    for landmark in mp_pose.PoseLandmark:
        positions[landmark] = (int(landmarks[landmark.value].x * frame_shape[1]),
                               int(landmarks[landmark.value].y * frame_shape[0]))
    return positions

def process_frame(image, class_name, frame_index, all_data_csv_writer, output_class_dir):
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        positions = get_joint_positions(landmarks, image.shape)
        draw_connections(image, positions)

        left_arm_angle = calculate_angle(positions[mp_pose.PoseLandmark.LEFT_SHOULDER],
                                         positions[mp_pose.PoseLandmark.LEFT_ELBOW],
                                         positions[mp_pose.PoseLandmark.LEFT_WRIST])
        right_arm_angle = calculate_angle(positions[mp_pose.PoseLandmark.RIGHT_SHOULDER],
                                          positions[mp_pose.PoseLandmark.RIGHT_ELBOW],
                                          positions[mp_pose.PoseLandmark.RIGHT_WRIST])
        left_foot_angle = calculate_angle(positions[mp_pose.PoseLandmark.LEFT_ANKLE],
                                          positions[mp_pose.PoseLandmark.LEFT_HEEL],
                                          positions[mp_pose.PoseLandmark.LEFT_FOOT_INDEX])
        right_foot_angle = calculate_angle(positions[mp_pose.PoseLandmark.RIGHT_ANKLE],
                                           positions[mp_pose.PoseLandmark.RIGHT_HEEL],
                                           positions[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX])
        distance_between_feet = calculate_distance(positions[mp_pose.PoseLandmark.LEFT_HEEL],
                                                   positions[mp_pose.PoseLandmark.RIGHT_HEEL])

        all_data_csv_writer.writerow([class_name, frame_index, left_foot_angle, right_foot_angle,
                                      distance_between_feet, left_arm_angle, right_arm_angle])

        processed_frame_path = os.path.join(output_class_dir, f'frame_{frame_index:04d}.jpg')
        cv2.imwrite(processed_frame_path, image)
    else:
        print(f"No landmarks detected in class {class_name}: frame {frame_index}")

def process_directory(input_dir, output_dir, all_data_csv_writer):
    for class_name in os.listdir(input_dir):
        class_dir = os.path.join(input_dir, class_name)
        if not os.path.isdir(class_dir):
            continue

        for subclass_name in os.listdir(class_dir):
            subclass_dir = os.path.join(class_dir, subclass_name)
            if not os.path.isdir(subclass_dir):
                continue

            output_class_dir = os.path.join(output_dir, class_name)
            os.makedirs(output_class_dir, exist_ok=True)

            frame_files = sorted([f for f in os.listdir(subclass_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            frame_index = 0

            for frame_file in frame_files:
                frame_path = os.path.join(subclass_dir, frame_file)
                frame = cv2.imread(frame_path)
                if frame is None:
                    print(f"Error reading frame {frame_file}")
                    continue

                process_frame(frame, class_name, frame_index, all_data_csv_writer, output_class_dir)
                frame_index += 1

def process_frames(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    all_data_csv_path = os.path.join(output_dir, 'all_class_pose_data.csv')

    with open(all_data_csv_path, 'w', newline='') as all_data_csv_file:
        all_data_csv_writer = csv.writer(all_data_csv_file)
        header = ['Class', 'Frame', 'Left_Foot_Angle', 'Right_Foot_Angle', 'Distance_Between_Feet',
                  'Left_Arm_Angle', 'Right_Arm_Angle']
        all_data_csv_writer.writerow(header)

        process_directory(input_dir, output_dir, all_data_csv_writer)

    print(f"All classes processed and data saved in '{all_data_csv_path}'")

input_dir = '/content/drive/MyDrive/dataset5/frames45'
output_dir = '/content/drive/MyDrive/dataset5/frm45_pro'
process_frames(input_dir, output_dir)


/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


No landmarks detected in class 1: frame 23
No landmarks detected in class 1: frame 25
No landmarks detected in class 1: frame 34
No landmarks detected in class 1: frame 45
No landmarks detected in class 1: frame 56
No landmarks detected in class 1: frame 59
No landmarks detected in class 1: frame 67
No landmarks detected in class 1: frame 78
No landmarks detected in class 1: frame 79
No landmarks detected in class 1: frame 89
No landmarks detected in class 1: frame 90
No landmarks detected in class 1: frame 91
No landmarks detected in class 1: frame 92
No landmarks detected in class 1: frame 93
No landmarks detected in class 1: frame 94
No landmarks detected in class 2: frame 0
No landmarks detected in class 2: frame 32
No landmarks detected in class 3: frame 0
No landmarks detected in class 3: frame 13
No landmarks detected in class 4: frame 31
No landmarks detected in class 7: frame 0
No landmarks detected in class 7: frame 17
No landmarks detected in class 7: frame 20
No landmarks d

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import os
import matplotlib.pyplot as plt

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def calculate_foot_angle(heel, toe):
    foot_vector = np.array(toe) - np.array(heel)
    floor_vector = np.array([1, 0])  # Horizontal line
    foot_vector_normalized = foot_vector / np.linalg.norm(foot_vector)
    floor_vector_normalized = floor_vector / np.linalg.norm(floor_vector)
    angle = np.arccos(np.clip(np.dot(foot_vector_normalized, floor_vector_normalized), -1.0, 1.0))
    return np.degrees(angle)

def draw_foot_info(image, heel, toe, angle, distance_text=None):
    cv2.circle(image, heel, 5, (255, 0, 0), -1)
    cv2.circle(image, toe, 5, (0, 255, 0), -1)
    cv2.line(image, heel, toe, (0, 0, 255), 2)
    text_pos = (heel[0], heel[1] - 10)
    cv2.putText(image, f"Angle: {angle:.2f} deg", text_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    if distance_text:
        cv2.putText(image, f"Dist: {distance_text:.2f} px", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

def get_landmarks(landmarks, frame_shape):
    left_heel = (int(landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x * frame_shape[1]),
                 int(landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y * frame_shape[0]))
    left_toe = (int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x * frame_shape[1]),
                int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y * frame_shape[0]))
    right_heel = (int(landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x * frame_shape[1]),
                  int(landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y * frame_shape[0]))
    right_toe = (int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x * frame_shape[1]),
                 int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y * frame_shape[0]))
    return left_heel, left_toe, right_heel, right_toe

input_dir = '/content/drive/MyDrive/dataset5/frames45'
output_dir = '/content/drive/MyDrive/dataset5/frm45_pro_test'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
all_data_csv_path = os.path.join(output_dir, 'all_class_foot_data.csv')

# Prepare a single CSV file for all data
with open(all_data_csv_path, 'w', newline='') as all_data_csv_file:
    all_data_csv_writer = csv.writer(all_data_csv_file)
    all_data_csv_writer.writerow(['Class', 'Frame', 'Left_Foot_Angle', 'Right_Foot_Angle', 'Distance_Between_Feet'])

    # Process each class directory
    for class_name in os.listdir(input_dir):
        class_dir = os.path.join(input_dir, class_name)
        if not os.path.isdir(class_dir):
            continue

        frame_files = sorted([f for f in os.listdir(class_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        frame_index = 0

        for frame_file in frame_files:
            frame_path = os.path.join(class_dir, frame_file)
            frame = cv2.imread(frame_path)
            if frame is None:
                print(f"Error reading frame {frame_file}")
                continue

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                left_heel, left_toe, right_heel, right_toe = get_landmarks(landmarks, frame.shape)
                distance = np.linalg.norm(np.array(left_heel) - np.array(right_heel))
                left_angle = calculate_foot_angle(left_heel, left_toe)
                right_angle = calculate_foot_angle(right_heel, right_toe)

                # Write data to the single CSV file
                all_data_csv_writer.writerow([class_name, frame_index, left_angle, right_angle, distance])
                draw_foot_info(image, left_heel, left_toe, left_angle, distance_text=distance)
                draw_foot_info(image, right_heel, right_toe, right_angle)

                # Save processed frame
                processed_frame_path = os.path.join(output_dir, f'{class_name}_frame_{frame_index:04d}.jpg')
                cv2.imwrite(processed_frame_path, image)

                plt.figure(figsize=(8, 8))
                plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                plt.title(f"Frame {frame_index} - {class_name}")
                plt.axis('off')
                plt.show()

                frame_index += 1

print(f"All classes processed and data saved in '{all_data_csv_path}'")


All classes processed and data saved in '/content/drive/MyDrive/dataset5/frm45_pro_test/all_class_foot_data.csv'


In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import os
import glob
import numpy as np

# Initialize MediaPipe pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=2)

# Function to process each image and extract landmarks
def process_image(image_path, class_name, output_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    landmarks = []

    if results.pose_landmarks:
        for landmark in results.pose_landmarks.landmark:
            landmarks.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])

        mp_drawing = mp.solutions.drawing_utils
        annotated_image = image.copy()
        mp_drawing.draw_landmarks(annotated_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        annotated_path = os.path.join(output_path, class_name, os.path.basename(image_path))
        os.makedirs(os.path.dirname(annotated_path), exist_ok=True)
        cv2.imwrite(annotated_path, annotated_image)

    return landmarks

def process_directory(input_path, output_path):
    all_data = []

    for class_dir in os.listdir(input_path):
        class_path = os.path.join(input_path, class_dir)
        if os.path.isdir(class_path):
            for image_path in glob.glob(class_path + '/*/*.jpg'):
                print(f"Processing {image_path}")
                landmarks = process_image(image_path, class_dir, output_path)
                if landmarks:
                    all_data.append([class_dir] + landmarks)

    if all_data:
        # Calculate columns based on actual data
        num_features = len(all_data[0]) - 1  # Subtracting the class column
        num_landmarks = num_features // 4
        print(f"Calculated number of features: {num_features}, Number of landmarks per feature: {num_landmarks}")

        columns = ['class'] + [f'x{i}_y{i}_z{i}_v{i}' for i in range(num_landmarks)]
        print(f"Creating DataFrame with columns: {len(columns)}")

        df = pd.DataFrame(all_data)
        excel_path = os.path.join(output_path, 'all_classes_data.xlsx')
        df.to_excel(excel_path, index=False)
        print(f"Data saved to {excel_path}")

input_path = '/content/drive/MyDrive/dataset5/frames00'
output_path = '/content/drive/MyDrive/dataset5/frm00_pro'

process_directory(input_path, output_path)


Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_55.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_56.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_57.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_58.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_59.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_60.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_61.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_62.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_63.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_64.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_65.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_66.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_67.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame

In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import os
import glob
import numpy as np

# Initialize MediaPipe pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=2)

# Function to process each image and extract landmarks and frame number
def process_image(image_path, class_name, output_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    landmarks = []

    if results.pose_landmarks:
        for landmark in results.pose_landmarks.landmark:
            landmarks.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])

        mp_drawing = mp.solutions.drawing_utils
        annotated_image = image.copy()
        mp_drawing.draw_landmarks(annotated_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        annotated_path = os.path.join(output_path, class_name, os.path.basename(image_path))
        os.makedirs(os.path.dirname(annotated_path), exist_ok=True)
        cv2.imwrite(annotated_path, annotated_image)

    # Extract frame number from image path
    frame_number = os.path.basename(image_path).split('.')[0].split('_')[-1]
    return [frame_number] + landmarks

def process_directory(input_path, output_path):
    all_data = []

    for class_dir in os.listdir(input_path):
        class_path = os.path.join(input_path, class_dir)
        if os.path.isdir(class_path):
            for image_path in glob.glob(class_path + '/*/*.jpg'):
                print(f"Processing {image_path}")
                frame_landmarks = process_image(image_path, class_dir, output_path)
                if frame_landmarks[1:]:  # Check if landmarks were extracted
                    all_data.append([class_dir] + frame_landmarks)

    if all_data:
        # Calculate columns based on actual data
        num_features = len(all_data[0]) - 2  # Subtracting the class and frame columns
        num_landmarks = num_features // 4
        print(f"Calculated number of features: {num_features}, Number of landmarks per feature: {num_landmarks}")

        columns = ['class', 'frame'] + [f'x{i}_y{i}_z{i}_v{i}' for i in range(num_landmarks)]
        print(f"Creating DataFrame with columns: {len(columns)}")

        df = pd.DataFrame(all_data)
        excel_path = os.path.join(output_path, 'all_classes_00_data.xlsx')
        df.to_excel(excel_path, index=False)
        print(f"Data saved to {excel_path}")

input_path = '/content/drive/MyDrive/dataset5/frames00'
output_path = '/content/drive/MyDrive/dataset5/frm45_pro_test'

process_directory(input_path, output_path)


Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_55.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_56.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_57.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_58.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_59.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_60.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_61.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_62.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_63.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_64.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_65.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_66.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame_67.jpg
Processing /content/drive/MyDrive/dataset5/frames00/1/1_00/frame

In [ ]:
import cv2
import pandas as pd
import mediapipe as mp
import os

# Initializing MediaPipe Pose class.
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Directory structure
input_dir = '/content/drive/MyDrive/dataset5/frames45'
output_dir = '/content/drive/MyDrive/dataset5/frm45_pro_test'

# Preparing the output CSV file
features_list = []

def process_images(input_dir, output_dir):
    print(f"Processing images in directory: {input_dir}")
    for class_dir in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_dir)
        if not os.path.isdir(class_path):
            continue
        print(f"Entering class directory: {class_path}")
        for sub_dir in os.listdir(class_path):
            sub_dir_path = os.path.join(class_path, sub_dir)
            if not os.path.isdir(sub_dir_path):
                continue
            for img_file in os.listdir(sub_dir_path):
                img_path = os.path.join(sub_dir_path, img_file)
                image = cv2.imread(img_path)
                if image is None:
                    print(f"Failed to load image: {img_path}")
                    continue

                print(f"Processing image: {img_path}")
                results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                if results.pose_landmarks:
                    landmarks = results.pose_landmarks.landmark
                    features = {
                        'class_name': class_dir,
                        'frame_name': img_file,
                         # Arm indices
                    'left_shoulder_x': landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x,
                    'left_shoulder_y': landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y,
                    'left_shoulder_z': landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].z,
                    'left_shoulder_visibility': landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].visibility,
                    'left_elbow_x': landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x,
                    'left_elbow_y': landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y,
                    'left_elbow_z': landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].z,
                    'left_elbow_visibility': landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].visibility,
                    'left_wrist_x': landmarks[mp_pose.PoseLandmark.LEFT_WRIST].x,
                    'left_wrist_y': landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y,
                    'left_wrist_z': landmarks[mp_pose.PoseLandmark.LEFT_WRIST].z,
                    'left_wrist_visibility': landmarks[mp_pose.PoseLandmark.LEFT_WRIST].visibility,
                    'right_shoulder_x': landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x,
                    'right_shoulder_y': landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y,
                    'right_shoulder_z': landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].z,
                    'right_shoulder_visibility': landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].visibility,
                    'right_elbow_x': landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].x,
                    'right_elbow_y': landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y,
                    'right_elbow_z': landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].z,
                    'right_elbow_visibility': landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].visibility,
                    'right_wrist_x': landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].x,
                    'right_wrist_y': landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].y,
                    'right_wrist_z': landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].z,
                    'right_wrist_visibility': landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].visibility,
                    # Foot indices
                    'left_heel_x': landmarks[mp_pose.PoseLandmark.LEFT_HEEL].x,
                    'left_heel_y': landmarks[mp_pose.PoseLandmark.LEFT_HEEL].y,
                    'left_heel_z': landmarks[mp_pose.PoseLandmark.LEFT_HEEL].z,
                    'left_heel_visibility': landmarks[mp_pose.PoseLandmark.LEFT_HEEL].visibility,
                    'left_foot_index_x': landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].x,
                    'left_foot_index_y': landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y,
                    'left_foot_index_z': landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].z,
                    'left_foot_index_visibility': landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].visibility,
                    'right_heel_x': landmarks[mp_pose.PoseLandmark.RIGHT_HEEL].x,
                    'right_heel_y': landmarks[mp_pose.PoseLandmark.RIGHT_HEEL].y,
                    'right_heel_z': landmarks[mp_pose.PoseLandmark.RIGHT_HEEL].z,
                    'right_heel_visibility': landmarks[mp_pose.PoseLandmark.RIGHT_HEEL].visibility,
                    'right_foot_index_x': landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].x,
                    'right_foot_index_y': landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y,
                    'right_foot_index_z': landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].z,
                    'right_foot_index_visibility': landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].visibility
                    }
                    features_list.append(features)

                    annotated_image = image.copy()
                    mp.solutions.drawing_utils.draw_landmarks(
                        annotated_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                    output_class_dir = os.path.join(output_dir, class_dir, sub_dir)
                    if not os.path.exists(output_class_dir):
                        os.makedirs(output_class_dir)
                    cv2.imwrite(os.path.join(output_class_dir, img_file), annotated_image)

process_images(input_dir, output_dir)

df = pd.DataFrame(features_list)
if not df.empty:
    df.to_csv(os.path.join(output_dir, 'features.csv'), index=False)
    print("CSV file has been written.")
else:
    print("No features were extracted. CSV file is empty.")

print("Processing complete.")


Processing images in directory: /content/drive/MyDrive/dataset5/frames45
Entering class directory: /content/drive/MyDrive/dataset5/frames45/1
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_0.jpg


/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_1.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_2.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_3.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_4.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_5.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_6.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_7.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_8.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_9.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_10.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_11.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_12.jpg
Processing image: /content/drive/MyDrive/dataset5

In [ ]:
import cv2
import pandas as pd
import mediapipe as mp
import os
import numpy as np

# Initializing MediaPipe Pose class.
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# Directory structure
input_dir = '/content/drive/MyDrive/dataset5/frames45'
output_dir = '/content/drive/MyDrive/dataset5/frm45_pro_test'

# Preparing the output CSV file
features_list = []

def calculate_3d_angle(p1, p2, p3):
    """Calculate the 3D angle between three points using vector math"""
    p1, p2, p3 = np.array(p1), np.array(p2), np.array(p3)
    v1, v2 = p1 - p2, p3 - p2
    cosine_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

def process_images(input_dir, output_dir):
    print(f"Processing images in directory: {input_dir}")
    for class_dir in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_dir)
        if not os.path.isdir(class_path):
            continue
        print(f"Entering class directory: {class_path}")
        for sub_dir in os.listdir(class_path):
            sub_dir_path = os.path.join(class_path, sub_dir)
            if not os.path.isdir(sub_dir_path):
                continue
            previous_landmarks = None  # Store landmarks from the previous frame for velocity calculations
            for img_file in sorted(os.listdir(sub_dir_path)):
                img_path = os.path.join(sub_dir_path, img_file)
                image = cv2.imread(img_path)
                if image is None:
                    print(f"Failed to load image: {img_path}")
                    continue

                print(f"Processing image: {img_path}")
                results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                if results.pose_landmarks:
                    landmarks = results.pose_landmarks.landmark
                    features = {'class_name': class_dir, 'frame_name': img_file}

                    # Calculate angles for elbows and wrists
                    if landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility > 0.5 and \
                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility > 0.5:
                        features['left_elbow_angle'] = calculate_3d_angle(
                            [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z],
                            [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].z],
                            [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].z]
                        )
                        features['right_elbow_angle'] = calculate_3d_angle(
                            [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z],
                            [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].z],
                            [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].z]
                        )

                    # Calculate velocities for selected major joints if previous landmarks are available
                    if previous_landmarks:
                        for landmark in [mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.RIGHT_ELBOW,
                                         mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.RIGHT_WRIST,
                                         mp_pose.PoseLandmark.LEFT_HEEL, mp_pose.PoseLandmark.RIGHT_HEEL,
                                         mp_pose.PoseLandmark.LEFT_FOOT_INDEX, mp_pose.PoseLandmark.RIGHT_FOOT_INDEX]:
                            idx = landmark.value
                            velocity = np.sqrt(
                                (landmarks[idx].x - previous_landmarks[idx].x) ** 2 +
                                (landmarks[idx].y - previous_landmarks[idx].y) ** 2 +
                                (landmarks[idx].z - previous_landmarks[idx].z) ** 2
                            )
                            features[f'{landmark.name.lower()}_velocity'] = velocity

                    features_list.append(features)
                    previous_landmarks = landmarks  # Update previous landmarks

                    # Optionally save annotated images
                    annotated_image = image.copy()
                    mp.solutions.drawing_utils.draw_landmarks(
                        annotated_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                    output_class_dir = os.path.join(output_dir, class_dir, sub_dir)
                    if not os.path.exists(output_class_dir):
                        os.makedirs(output_class_dir)
                    cv2.imwrite(os.path.join(output_class_dir, img_file), annotated_image)

    # Save all features to a CSV file
    df = pd.DataFrame(features_list)
    if not df.empty:
        df.to_csv(os.path.join(output_dir, 'features45.csv'), index=False)
        print("CSV file has been written.")
    else:
        print("No features were extracted. CSV file is empty.")

process_images(input_dir, output_dir)

print("Processing complete.")


Processing images in directory: /content/drive/MyDrive/dataset5/frames45
Entering class directory: /content/drive/MyDrive/dataset5/frames45/1
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_0.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_1.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_10.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_11.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_12.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_13.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_14.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_15.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_16.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1/1_l45/frame_17.jpg
Processing image: /content/drive/MyDrive/dataset5/frames45/1